## Imports

In [1]:
import torch
import shutil
import random
import glob

import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image

from torch import optim
from torch import nn

from matplotlib.patches import Rectangle

import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision.datasets import ImageNet
from torchvision import transforms
from torchvision import models

import torchvision


import matplotlib.pyplot as plt

## Data sorting

In [2]:
######## CARS #########
f1  = open("class_string.txt", "r").read().split("\n")
f2  = open("fname_string.txt", "r").read().split("\n")
f3 = open("bbox.txt", "r").read().split("\n")
totbbox = []
for ind, line in enumerate(f3):
    f1[ind] = int(f1[ind])
    bbox = line.split("\t") # x1,y1,x2,y2
    bbox[0] = float(bbox[0])
    bbox[1] = float(bbox[1])
    bbox[2] = float(bbox[2])
    bbox[3] = float(bbox[3])
    totbbox.append(bbox)
fin_list = []
for i in range(len(f2)):
    fin_list.append([f2[i],f1[i],totbbox[i][0],totbbox[i][1],totbbox[i][2],totbbox[i][3]])
df = pd.DataFrame(fin_list, columns=['image_id','labels', 'x0','y0','x1','y1'])


In [4]:
######## License plates ##########
f4 = open("lp_bb.txt", "r").read().split("\n")
totbbox = []
for line in f4:
    if(line==""):
        break
    bbox = line.split("\t") # x1,y1,x2,y2
    bbox[0] = float(bbox[0])
    bbox[1] = float(bbox[1])
    bbox[2] = float(bbox[2])
    bbox[3] = float(bbox[3])
    totbbox.append(bbox)
fin_list = []
for i in range(len(f4)-1):
    fin_list.append(["Cars{}.png".format(i),1,totbbox[i][0],totbbox[i][1],totbbox[i][2],totbbox[i][3]])
df_LP = pd.DataFrame(fin_list, columns=['image_id','labels', 'x0','y0','x1','y1'])

## Data Handler


In [5]:
class CarDataset(Dataset):
    def __init__(self, df, image_dir, transforms= None):
        super().__init__()
        
        self.df = df
        self.image_ids = self.df['image_id']
        self.image_dir = image_dir
        self.labels = self.df['labels']
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image_id"]==image_id]
        
        img = Image.open(self.image_dir/image_id).convert("RGB")
        img = transforms.ToTensor()(img)
        
        boxes = records[["x0","y0", "x1", "y1"]].values
        boxes = torch.tensor(boxes)
        boxes = boxes.type(torch.FloatTensor)
        labels = records["labels"].values
        labels = torch.tensor(labels)
        labels = labels.type(torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        return img, target
                            
                                  
                                  
    def __len__(self):
        return self.image_ids.shape[0]
    
def collate_fn(batch):
    return tuple(zip(*batch))  
        
dataset = CarDataset(df, Path.cwd() / "cars_train", f1) #"train_all"
dataset_LP = CarDataset(df_LP,Path.cwd() / "LPtrain/images",[1 for i in range(433)])        
dataloaded = DataLoader(dataset, batch_size=2, collate_fn = collate_fn)  
dataloaded_LP = DataLoader(dataset_LP, batch_size=2, collate_fn = collate_fn)

## Create model

In [6]:
###### CAR ########
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = models.detection.fasterrcnn_resnet50_fpn(pretrained = True, pretrained_backbone = True)#, num_classes = 197)

classes = 197
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

load_old_model = True

if(load_old_model):
    model.load_state_dict(torch.load(Path.cwd() / "one_epoch"))



In [ ]:
###### License plate ########
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model_LP = models.detection.fasterrcnn_resnet50_fpn(pretrained = True, pretrained_backbone = True)#, num_classes = 197)

classes = 2
in_features = model_LP.roi_heads.box_predictor.cls_score.in_features
model_LP.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_LP.to(device)

load_old_model_LP = False

if(load_old_model_LP):
    model_LP.load_state_dict(torch.load(Path.cwd() / ""))


## Training


In [7]:
###### CAR ######
model.train()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 3
for epoch in range(num_epochs):
    for i,(b_x, b_y) in enumerate(dataloaded):
        b_x = [item.to(device) for item in b_x]
        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
        loss_dict=model(b_x,b_y)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        if(i%100 == 0):
            print(losses)
    lr_scheduler.step()
    print(epoch)

save_model = False
if(save_model):
    torch.save(model.state_dict(), "Model_1")
    

tensor(0.1054, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1635, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1025, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1727, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1467, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0980, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1191, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1824, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1336, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1016, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1087, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1765, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1975, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1286, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1064, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1506, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1327, device='cuda:0', grad_fn=<AddBack

In [ ]:
model_LP.train()

params = [p for p in model_LP.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 1
for epoch in range(num_epochs):
    for i,(b_x, b_y) in enumerate(dataloaded_LP):
        b_x = [item.to(device) for item in b_x]
        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
        loss_dict=model_LP(b_x,b_y)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        if(i%10 == 0):
            print(losses)
    lr_scheduler.step()
    print(epoch)

save_model_LP = False
if(save_model_LP):
    torch.save(model_LP.state_dict(), "Model_1")

## Evaluation

In [39]:
def eval_and_plot_image(model, df, indx,plot_image=False):
    model = model.eval()
    image, target = df[indx]
    images = torch.stack([image]+[image]).to(device)
    with torch.no_grad():
        out=model(images)
    if(plot_image):
        fig, ax = plt.subplots()
        ax.imshow(image.permute(1, 2, 0))
        boxes=[]
        box = out[0]['boxes'][0]
        width=(box[2]-box[0])
        height=(box[3]-box[1])
        ax.add_patch(Rectangle((box[0],box[1]),width,height,fill = False))
    keep = torchvision.ops.nms(out[0]['boxes'], out[0]['scores'], 0.3)
    for indx in keep:
        s = "Label: {}/{}, score: {}".format(out[0]['labels'][indx],target['labels'][0],out[0]['scores'][indx])
        if(plot_image):
            ax.text(out[0]['boxes'][indx][0],out[0]['boxes'][indx][1],s)
        print(s)
        if(out[0]['scores'][indx] > 0.5):
            return out[0]['labels'][indx]
        else:
            return 0
#eval_and_plot_image(model, dataset, 29,True)
indices=[]
for i in range(8000):        
    lab = eval_and_plot_image(model, dataset, i)
    indices.append(lab)
with open('50proc.txt', 'w') as f:
    for item in indices:
        f.write("{}\n".format(item))

Label: 105/14, score: 0.20009712874889374
Label: 3/3, score: 0.10108136385679245
Label: 17/91, score: 0.3427751958370209
Label: 168/134, score: 0.4903654456138611
Label: 1/106, score: 0.1932922899723053
Label: 6/123, score: 0.30618563294410706
Label: 73/89, score: 0.14408716559410095
Label: 153/96, score: 0.39070165157318115
Label: 164/167, score: 0.08449730277061462
Label: 50/58, score: 0.5568570494651794
Label: 138/49, score: 0.11994577944278717
Label: 95/186, score: 0.14851579070091248
Label: 85/135, score: 0.06975668668746948
Label: 181/85, score: 0.10045269131660461
Label: 117/193, score: 0.1418011337518692
Label: 6/172, score: 0.8336222171783447
Label: 66/14, score: 0.15535873174667358
Label: 85/73, score: 0.1799481064081192
Label: 192/192, score: 0.4982333779335022
Label: 151/57, score: 0.2635814845561981
Label: 29/79, score: 0.09415995329618454
Label: 55/36, score: 0.3734404742717743
Label: 62/120, score: 0.20775625109672546
Label: 168/170, score: 0.2962509095668793
Label: 194/

Label: 191/191, score: 0.3260389268398285
Label: 17/16, score: 0.4384436011314392
Label: 153/12, score: 0.1785830706357956
Label: 75/90, score: 0.3610093295574188
Label: 66/188, score: 0.1591961532831192
Label: 151/128, score: 0.2598881721496582
Label: 150/23, score: 0.139870747923851
Label: 151/11, score: 0.21577687561511993
Label: 153/93, score: 0.5002450942993164
Label: 150/8, score: 0.20900921523571014
Label: 63/63, score: 0.11459233611822128
Label: 128/173, score: 0.11967900395393372
Label: 96/176, score: 0.1011975109577179
Label: 6/72, score: 0.1723526567220688
Label: 17/195, score: 0.4113839864730835
Label: 153/114, score: 0.1294119656085968
Label: 66/72, score: 0.17576242983341217
Label: 119/119, score: 0.2697904407978058
Label: 172/18, score: 0.18266713619232178
Label: 77/77, score: 0.40741902589797974
Label: 17/18, score: 0.27540215849876404
Label: 156/107, score: 0.173757404088974
Label: 105/10, score: 0.1076372042298317
Label: 196/196, score: 0.9195399284362793
Label: 117/9

Label: 13/23, score: 0.20705565810203552
Label: 17/40, score: 0.2597895562648773
Label: 164/26, score: 0.0955309197306633
Label: 181/181, score: 0.1296904981136322
Label: 166/88, score: 0.411462664604187
Label: 17/17, score: 0.2915786802768707
Label: 168/110, score: 0.07156281173229218
Label: 55/180, score: 0.18844370543956757
Label: 102/10, score: 0.11758673936128616
Label: 75/75, score: 0.23489831387996674
Label: 59/59, score: 0.2695194184780121
Label: 150/19, score: 0.5793443918228149
Label: 181/189, score: 0.07359711825847626
Label: 117/5, score: 0.186209037899971
Label: 105/14, score: 0.11631429940462112
Label: 176/176, score: 0.3468027710914612
Label: 59/80, score: 0.19119489192962646
Label: 75/86, score: 0.22036588191986084
Label: 55/55, score: 0.8439185619354248
Label: 153/112, score: 0.4233565330505371
Label: 131/50, score: 0.18698787689208984
Label: 160/57, score: 0.3363158404827118
Label: 73/73, score: 0.18688859045505524
Label: 168/99, score: 0.1729152649641037
Label: 153/1

Label: 75/69, score: 0.48445457220077515
Label: 123/123, score: 0.4959046542644501
Label: 75/75, score: 0.4438038468360901
Label: 29/28, score: 0.16167250275611877
Label: 95/186, score: 0.21509848535060883
Label: 24/20, score: 0.21779674291610718
Label: 117/63, score: 0.4809061884880066
Label: 150/20, score: 0.19260820746421814
Label: 17/16, score: 0.21635712683200836
Label: 66/77, score: 0.09272455424070358
Label: 151/160, score: 0.25457799434661865
Label: 160/98, score: 0.3575676381587982
Label: 59/59, score: 0.2841360867023468
Label: 151/150, score: 0.5868186354637146
Label: 153/123, score: 0.39132586121559143
Label: 119/145, score: 0.43210819363594055
Label: 6/80, score: 0.31637999415397644
Label: 79/79, score: 0.2713610529899597
Label: 119/169, score: 0.2037261426448822
Label: 127/115, score: 0.08531169593334198
Label: 75/75, score: 0.32965022325515747
Label: 6/98, score: 0.304363876581192
Label: 1/124, score: 0.24798135459423065
Label: 50/89, score: 0.2292809784412384
Label: 6/12

Label: 93/24, score: 0.14855042099952698
Label: 117/139, score: 0.37436941266059875
Label: 51/51, score: 0.11746826767921448
Label: 126/184, score: 0.4182656407356262
Label: 191/191, score: 0.3330934941768646
Label: 107/27, score: 0.14647306501865387
Label: 6/172, score: 0.4295279085636139
Label: 92/107, score: 0.17674313485622406
Label: 151/103, score: 0.4082299470901489
Label: 151/163, score: 0.20366784930229187
Label: 150/24, score: 0.10675821453332901
Label: 166/166, score: 0.45289793610572815
Label: 66/190, score: 0.09878618270158768
Label: 92/92, score: 0.34009864926338196
Label: 142/121, score: 0.09966710954904556
Label: 119/119, score: 0.2905866503715515
Label: 6/160, score: 0.653037965297699
Label: 119/111, score: 0.22011587023735046
Label: 117/117, score: 0.7191590666770935
Label: 17/68, score: 0.1209990382194519
Label: 142/139, score: 0.1909949630498886
Label: 6/82, score: 0.35122260451316833
Label: 187/137, score: 0.09399177134037018
Label: 153/57, score: 0.2867511212825775

...

Label: 111/111, score: 0.23451706767082214
Label: 117/178, score: 0.23360927402973175
Label: 62/62, score: 0.15595251321792603
Label: 24/176, score: 0.348028302192688
Label: 119/64, score: 0.5361626148223877
Label: 119/119, score: 0.5833033323287964
Label: 24/40, score: 0.12187469750642776
Label: 66/28, score: 0.12239012122154236
Label: 166/108, score: 0.19413509964942932
Label: 29/42, score: 0.20557689666748047
Label: 119/111, score: 0.3291351795196533
Label: 50/110, score: 0.38262826204299927
Label: 166/169, score: 0.4578664004802704
Label: 66/66, score: 0.3491801619529724
Label: 17/108, score: 0.23914429545402527
Label: 150/104, score: 0.1405177265405655
Label: 166/154, score: 0.31358441710472107
Label: 60/60, score: 0.4725034534931183
Label: 17/73, score: 0.13639017939567566
Label: 166/58, score: 0.21914663910865784
Label: 66/42, score: 0.20626430213451385
Label: 66/7, score: 0.22710159420967102
Label: 85/85, score: 0.5496873259544373
Label: 6/6, score: 0.8636981844902039
Label: 16

Label: 123/123, score: 0.4868822991847992
Label: 138/140, score: 0.29656341671943665
Label: 119/62, score: 0.23592665791511536
Label: 128/128, score: 0.11741600185632706
Label: 66/7, score: 0.21284854412078857
Label: 142/142, score: 0.26880356669425964
Label: 66/43, score: 0.07480626553297043
Label: 36/36, score: 0.19499129056930542
Label: 85/85, score: 0.4709523022174835
Label: 32/174, score: 0.09372363984584808
Label: 138/138, score: 0.23933246731758118
Label: 128/187, score: 0.10000820457935333
Label: 151/163, score: 0.15336163341999054
Label: 192/192, score: 0.5484234094619751
Label: 1/74, score: 0.3039543330669403
Label: 151/151, score: 0.29725292325019836
Label: 142/33, score: 0.11024795472621918
Label: 164/4, score: 0.07891079783439636
Label: 157/157, score: 0.2277381271123886
Label: 128/35, score: 0.08372456580400467
Label: 151/151, score: 0.3208475112915039
Label: 160/137, score: 0.07706423103809357
Label: 95/195, score: 0.1052674725651741
Label: 153/93, score: 0.2026024311780

Label: 102/56, score: 0.21822351217269897
Label: 173/141, score: 0.11564666032791138
Label: 117/50, score: 0.36066463589668274
Label: 6/72, score: 0.31977036595344543
Label: 119/143, score: 0.4243854880332947
Label: 181/181, score: 0.13328897953033447
Label: 192/192, score: 0.3996216356754303
Label: 119/90, score: 0.3467875123023987
Label: 29/24, score: 0.11444433778524399
Label: 50/76, score: 0.16661684215068817
Label: 97/97, score: 0.3584185838699341
Label: 17/90, score: 0.19951294362545013
Label: 188/30, score: 0.10370617359876633
Label: 166/174, score: 0.31317830085754395
Label: 6/97, score: 0.25347959995269775
Label: 151/151, score: 0.45212724804878235
Label: 79/79, score: 0.37316131591796875
Label: 24/24, score: 0.4216230809688568
Label: 66/67, score: 0.12742486596107483
Label: 55/80, score: 0.7241984605789185
Label: 85/85, score: 0.4132547974586487
Label: 17/17, score: 0.32984766364097595
Label: 17/156, score: 0.3379318416118622
Label: 155/155, score: 0.15049105882644653
Label: 

Label: 153/179, score: 0.5439744591712952
Label: 1/125, score: 0.6747663617134094
Label: 66/66, score: 0.6321813464164734
Label: 17/156, score: 0.49846383929252625
Label: 161/161, score: 0.29107391834259033
Label: 24/167, score: 0.321572482585907
Label: 153/55, score: 0.8937407732009888
Label: 1/124, score: 0.45833688974380493
Label: 29/29, score: 0.5920175313949585
Label: 29/4, score: 0.10473203659057617
Label: 1/145, score: 0.4787474572658539
Label: 93/42, score: 0.11783745139837265
Label: 164/129, score: 0.16451548039913177
Label: 73/73, score: 0.26889774203300476
Label: 62/120, score: 0.11223911494016647
Label: 6/158, score: 0.4047936201095581
Label: 36/36, score: 0.3140193819999695
Label: 168/140, score: 0.11737710982561111
Label: 153/123, score: 0.6149754524230957
Label: 37/94, score: 0.13663437962532043
Label: 30/77, score: 0.05962622910737991
Label: 36/36, score: 0.2562081515789032
Label: 150/14, score: 0.15402285754680634
Label: 151/46, score: 0.301802396774292
Label: 6/33, sc

Label: 93/157, score: 0.1925898641347885
Label: 166/131, score: 0.2104705572128296
Label: 62/62, score: 0.3762138783931732
Label: 171/158, score: 0.1193026453256607
Label: 128/128, score: 0.16730152070522308
Label: 166/143, score: 0.35100388526916504
Label: 37/89, score: 0.10212163627147675
Label: 66/44, score: 0.17680241167545319
Label: 191/191, score: 0.29847079515457153
Label: 95/120, score: 0.1029566079378128
Label: 117/49, score: 0.22213047742843628
Label: 128/128, score: 0.14265058934688568
Label: 6/77, score: 0.3440021872520447
Label: 152/56, score: 0.13374508917331696
Label: 117/117, score: 0.7207264304161072
Label: 66/51, score: 0.14837180078029633
Label: 126/181, score: 0.13059961795806885
Label: 192/192, score: 0.4130662977695465
Label: 157/157, score: 0.32952362298965454
Label: 129/2, score: 0.07927900552749634
Label: 66/128, score: 0.3980133831501007
Label: 117/129, score: 0.2106136977672577
Label: 75/69, score: 0.3311102092266083
Label: 42/42, score: 0.13224826753139496
L

In [ ]:
import xml.dom.minidom
def get_BB_xml(path):
    document = xml.dom.minidom.parse(path)
    x1 = document.getElementsByTagName("xmin")[0]
    x1=x1.childNodes[0]
    x1=x1.nodeValue
    y1 = document.getElementsByTagName("ymin")[0]
    y1=y1.childNodes[0]
    y1=y1.nodeValue
    x2 = document.getElementsByTagName("xmax")[0]
    x2=x2.childNodes[0]
    x2=x2.nodeValue
    y2 = document.getElementsByTagName("ymax")[0]
    y2=y2.childNodes[0]
    y2=y2.nodeValue
    return torch.tensor([float(x1),float(y1),float(x2),float(y2)],dtype=torch.float32)
bbox=[]
for i in range(433):
    bbox.append(get_BB_xml("LPtrain/annotations/Cars{}.xml".format(i)))
with open('lp_bb.txt', 'w') as f:
    for item in bbox:
        f.write("{}\t{}\t{}\t{}\n".format(item[0],item[1],item[2],item[3]))

